# Real time pen detection

In [21]:
# download Real-time_pen_detection repository
!git clone https://github.com/InnovationLab-Top/Real-time_pen_detection.git

Cloning into 'Real-time_pen_detection'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 81 (delta 8), reused 75 (delta 2), pack-reused 0
Unpacking objects: 100% (81/81), done.


In [ ]:
# download YOLOX
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git
%cd /content/YOLOX
!pip3 install -U pip && pip3 install -r requirements.txt
!pip3 install -v -e .
%cd ..

In [9]:
# add num_classes in yolox_s.py for the training
with open('/content/YOLOX/exps/default/yolox_s.py', 'r') as file_src:
  lines = file_src.readlines()
with open('/content/YOLOX/exps/default/yolox_s.py', 'w+') as file_dst:
  lines[14] += '        self.num_classes = 1\n'
  file_dst.write("".join(lines))

# change classes in /content/YOLOX/yolox/data/datasets/coco_classes.py
with open('/content/YOLOX/yolox/data/datasets/coco_classes.py', 'r') as file_src:
  lines = file_src.readlines()
with open('/content/YOLOX/yolox/data/datasets/coco_classes.py', 'w+') as file_dst:
  lines[3] = '"""\n'
  lines[85] = ')"""\n'
  lines[85] += 'COCO_CLASSES = ("pen",)\n'  #change COCO_CLASSES with pen item (COMMA IS MANDATORY)
  file_dst.write("".join(lines))

## If you want to train the model



In [ ]:
# download YOLO-to-COCO format converter
!git clone https://github.com/Taeyoung96/Yolo-to-COCO-format-converter.git
# May need to change in the future if Colab no longer uses CUDA 11.0
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html
# DISCLAIMER: this installation requires a little bit time (5 minutes)

In [ ]:
# install NVIDIA Apex
# DISCLAIMER: this installation requires a lot of time (about 10 minutes)
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd .. 

In [ ]:
# install PyCocoTools
!pip3 install cython; pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
# create directories for COCO dataset format
!mkdir /content/YOLOX/datasets/COCO/
!mkdir /content/YOLOX/datasets/COCO/train2017/
!mkdir /content/YOLOX/datasets/COCO/annotations/
!mkdir /content/YOLOX/datasets/COCO/val2017/
!mkdir /content/YOLOX/datasets/COCO/unlabeled2017/
!mkdir /content/YOLOX/datasets/COCO/train2017/YOLO_annotations/
!mkdir /content/YOLOX/datasets/COCO/val2017/YOLO_annotations/

In [ ]:
# copying images from Real-time_pen_detection repository to train2017 and val2017
! cp /content/Real-time_pen_detection/Images_resized/*  /content/YOLOX/datasets/COCO/train2017/
! cp /content/Real-time_pen_detection/Images_resized/*  /content/YOLOX/datasets/COCO/val2017/
# copying labels from Real-time_pen_detection repository to train2017 and val2017
! cp /content/Real-time_pen_detection/labels_txt/*  /content/YOLOX/datasets/COCO/train2017/YOLO_annotations/
! cp /content/Real-time_pen_detection/labels_txt/*  /content/YOLOX/datasets/COCO/val2017/YOLO_annotations/

In [ ]:
# change main.py in Yolo-to-COCO-format-converter
with open('/content/Yolo-to-COCO-format-converter/main.py', 'r') as file_src:
  lines = file_src.readlines()
with open('/content/Yolo-to-COCO-format-converter/main.py', 'w+') as file_dst:
  lines[17] = 'YOLO_DARKNET_SUB_DIR = "YOLO_annotations"\n'  #folder in train/val2017 for annotations
  lines[18] = "'''\n"
  lines[26] = "'''\n"
  lines[27] = 'classes = ["pen"]\n'  #change classes to pen
  lines[71] = '                int(label_line.split()[0])\n'  #delete +1 to get category_id = 0
  lines[205] = '    output_path = "/content/YOLOX/datasets/COCO/annotations/" + output_name\n'  #change output path
  lines[221] = '                "id": index,  \n'  #delete +1 to get category_id = 0
  file_dst.write("".join(lines))

In [ ]:
# convert Yolo to COCO format
!python /content/Yolo-to-COCO-format-converter/main.py --yolo-subdir --path /content/YOLOX/datasets/COCO/train2017 --output instances_train2017.json
!python /content/Yolo-to-COCO-format-converter/main.py --yolo-subdir --path /content/YOLOX/datasets/COCO/val2017 --output instances_val2017.json

In [ ]:
# download pretrained weights on COCO dataset to initialize model weights
%cd /content/
!wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_s.pth

PRETRAINED_PATH = '/content/yolox_s.pth'

In [ ]:
# training
MODEL_PATH = '/content/YOLOX/exps/default/yolox_s.py'
TRAIN_PATH = '/content/YOLOX/tools/train.py'

!python {TRAIN_PATH} -f {MODEL_PATH} -d 1 -b 8 --fp16 -o -c {PRETRAINED_PATH}

In [ ]:
# test on an image
DEMO_PATH = '/content/YOLOX/tools/demo.py'
TEST_IMAGE_PATH = '/content/Real-time_pen_detection/Test_frap.jpg'
WEIGHTS_PATH = '/content/YOLOX_outputs/yolox_s/latest_ckpt.pth'

!python {DEMO_PATH} image -f {MODEL_PATH} -c {WEIGHTS_PATH} --path {TEST_IMAGE_PATH} --conf 0.4 --nms 0.45 --save_result --device gpu

## If you want to use pretrained model

In [11]:
import torch
%cd /content/YOLOX/

/content/YOLOX


In [ ]:
# loading model
model = torch.load('/content/Real-time_pen_detection/model.pth')
model.cuda()
model.eval()

## Visualize real time pen detection

In [22]:
# change original yolox demo.py file to modified one
!cp /content/Real-time_pen_detection/demo.py /content/YOLOX/tools/demo.py

In [13]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt

# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [34]:
# import functions from demo.py module
%cd /content/YOLOX/
from tools.demo import *
from yolox.data.datasets import COCO_CLASSES
from yolox.exp.build import *

/content/YOLOX


In [35]:
# define the model predictor
predictor = Predictor(model, get_exp_by_file('/content/YOLOX/exps/default/yolox_s.py'), COCO_CLASSES, device = 'gpu')

In [65]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
detections = []
while True:
  js_reply = video_frame(label_html, bbox)
  if not js_reply:
      break

  # convert JS response to OpenCV Image
  frame = js_to_image(js_reply["img"])
  # create transparent overlay for bounding box
  bbox_array = np.zeros([480,640,4], dtype=np.uint8)

  # call our yolox_s on video frame
  outputs, img_info = predictor.inference_raw(frame)
  detections = predictor.visual_raw(outputs[0], img_info) #questa linea è costata un oreo a FraV

  # get labels and bboxes only if there's at least one detection (if no object is detected, detections will be a np.ndarray, else it will be a tuple of torch tensors)
  if type(detections) != np.ndarray:
    for i, _ in enumerate(detections[0]):

      bbox = detections[0][i]
      label = "pen"
      confidence = detections[2][i]

      left = int(bbox[0])
      top = int(bbox[1])
      right = int(bbox[2]) 
      bottom = int(bbox[3])

      if confidence > 0.5:
        bbox_array = cv2.rectangle(bbox_array, (left, top), (right, bottom), (255, 0, 0), 2)
        bbox_array = cv2.putText(bbox_array, "{} [{:.2f}]".format( label, float(confidence) ), 
                                (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
  bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
  # convert overlay of bbox into bytes
  bbox_bytes = bbox_to_bytes(bbox_array)
  # update bbox so next frame gets new overlay
  bbox = bbox_bytes
  

<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored

In [64]:
print(((type(detections))))

<class 'numpy.ndarray'>


In [ ]:
predictor = Predictor(model, get_exp_by_file(MODEL_PATH), COCO_CLASSES, device = 'gpu')